# 6.1 텍스트 데이터 다루기

## 6.1.1 단어와 문자의 원-핫 인코딩

In [0]:
# 단어 수준의 원-핫 인코딩하기
import numpy as np

samples = ['The cat sat on the mat.', 'The dog ate my homework.']  # 초기 데이터: 각원소가 샘플

token_index = {}  # 데이터에 있는 모든 토큰의 인덱스를 구축
for sample in samples:
  for word in sample.split():   # split() 메서드를 사용하여 샘플을 토큰으로 나눔
    if word not in token_index:
      token_index[word] = len(token_index) + 1  # 단어마다 고유한 인덱스를 할당합니다. 인덱스 0은 사용하지 않는다  

max_length = 10  # 샘플을 벡터로 변환합니다. 각 샘플에서 max_length까지 단어만 사용
results = np.zeros(shape=(len(samples),
                          max_length,
                          max(token_index.values()) + 1))   # 경과를 저장할 배열
for i, sample in enumerate(samples):
  for j, word in list(enumerate(sample.split()))[:max_length]:
    index = token_index.get(word)
    results[i, j, index] = 1.

In [0]:
# 문자 수준 원-핫 인코딩하기
import string

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
characters = string.printable  # 출력 가능한 모든 아스키 문자
token_index = dict(zip(characters, range(1, len(characters) + 1)))

max_length = 50
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
  for j, character in enumerate(sample):
    index = token_index.get(character)
    results[i, j, index] = 1.

In [0]:
# 캐라스를 사용한 단어 수준의 원-핫 인코딩하기
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

tokenizer = Tokenizer(num_words=1000) # 가장 빈도가 높은 1000개의 단어만 선택하도록 Tokenizer 객체를 만듬
tokenizer.fit_on_texts(samples)  # 단어 인덱스를 구축

sequences = tokenizer.texts_to_sequences(samples)  # 문자열을 정수 인덱스의 리스트로 변환

one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')  # 직접 원-핫 이진 벡터 표현을 얻을수 있음. 원-핫 인코딩 외에 다른 벡터화 방법들도 제공

word_index = tokenizer.word_index  # 계산된 단어 인덱스를 구함
print('%s개의 고유한 토큰을 찾았습니다.' % len(word_index))

9개의 고유한 토큰을 찾았습니다.


In [0]:
# 해싱 기법을 사용한 단어 수준의 원-핫 인코딩하기
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

dimensionality = 1000 # 단어를 크기가 1000인 벡터로 저장. 1000개의 단어가 있다면 해싱 충돌이 늘어나고 인코딩의 정확도가 감소될것
max_length = 10

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
  for j, word in list(enumerate(sample.split()))[:max_length]:
    index = abs(hash(word)) % dimensionality # 단어를 해싱하여 0과 1000사이의 랜덤한 정수 인덱스로 변환
    results[i, j, index] = 1.